## Test script to use during refactor of scattering code



In [1]:
from sn_camels.models.models_factory import baseModelFactory, topModelFactory
from sn_camels.models.sn_hybrid_models import sn_HybridModel
from sn_camels.models.camels_models import get_architecture 
from sn_camels.camels.camels_dataset import *

In [2]:
## Check if CUDA available
if torch.cuda.is_available():
    print("CUDA Available")
    device = torch.device('cuda')
    use_cuda=True
else:
    print('CUDA Not Available')
    device = torch.device('cpu')
    use_cuda=False

CUDA Not Available


In [3]:
camels_path="/home/chris/Data/CAMELs"
fparams    = camels_path+"/params_IllustrisTNG.txt"
fmaps      = [
              camels_path+"/maps_Mtot.npy"
             ]

In [4]:
## dataloader options
seed=123,

batch_size=10
splits=1
fmaps_norm=[None]
num_workers=1
rot_flip_in_mem=True
channels=1
features=1

train_loader = create_dataset_multifield('train', seed, fmaps, fparams, batch_size, splits, fmaps_norm,
                                                 num_workers=num_workers, rot_flip_in_mem=rot_flip_in_mem, verbose=True)

Found 1 channels
Reading data...
6.054e+09 < F(all|orig) < 2.176e+15
9.782 < F(all|resc)  < 15.338
-2.696 < F(all|norm) < 8.631
Channel 0 contains 7200 maps
-2.696 < F < 8.631



In [5]:
scatteringBase = baseModelFactory( #creat scattering base model
    architecture='scattering',
    J=2,
    N=256,
    M=256,
    channels=channels,
    max_order=2,
    initialization="Random",
    seed=234,
    learnable=True,
    lr_orientation=0.03,
    lr_scattering=0.03,
    skip=True,
    split_filters=True,
    filter_video=False,
    subsample=4,
    device=device,
    use_cuda=use_cuda,
    plot=False
)

## Now create a network to follow the scattering layers
## can be MLP, linear, or cnn at the moment
## (as in https://github.com/bentherien/ParametricScatteringNetworks/ )
top = topModelFactory( #create cnn, mlp, linear_layer, or other
    base=scatteringBase,
    architecture="linear_layer",
    num_classes=features,
    width=3,
    average=True,
    use_cuda=use_cuda
)

hybridModel = sn_HybridModel(scatteringBase=scatteringBase, top=top, use_cuda=use_cuda)
model=hybridModel
model.to(device=device)


(256, 256)
(128, 128)


/home/chris/.local/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/chris/.local/lib/python3.8/site-packages/sn_camels-0.0.1-py3.8.egg/sn_camels/scattering/create_filters.py:283: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2981.)


sn_HybridModel(
  (scatteringBase): sn_ScatteringBase()
  (top): sn_LinearLayer(
    (fc1): Linear(in_features=73, out_features=1, bias=True)
    (bn0): BatchNorm1d(73, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [9]:
scatteringBase.phi[0].shape

(256, 256)

In [11]:
x,y=next(iter(train_loader))
model.to(device=device)
x=x.to(device=device)

In [12]:
## Check that we produce the correct shape
scatteringBase.forward(x).shape

TypeError: The input should be complex (i.e. last dimension is 2).

In [13]:
scatteringBase.backend.fft(x)

TypeError: The input should be complex (i.e. last dimension is 2).

In [14]:
complex_maps = x.new_zeros(x.shape + (2,))
complex_maps[..., 0] = x
scatteringBase.backend.fft(complex_maps).shape

torch.Size([10, 1, 256, 256, 2])

In [15]:
scatteringBase.phi[0].shape

## THE KYMATIO PHI GETS RESHAPED TO (256,256,1), WHICH IS CAUSING THE ISSUES

(256, 256)

In [ ]:
filter_1=2
filter_2=5
order2_index=17+filter_1*16+filter_2

cdm_map_num=7 # between 0 and 10

In [ ]:
plt.figure(figsize=(32,22))
plt.subplot(1, 3, 1)
plt.title("Original field")
plt.imshow(np.log10(maps[cdm_map_num]))
plt.subplot(1, 3, 2)
plt.title("Wavelet filter")
plt.imshow(np.fft.fftshift(np.fft.ifft2(scatteringBase.scattering.psi[filter_1][0].squeeze().cpu().detach().numpy())).real)
plt.subplot(1, 3, 3)
plt.title("First order convolved field")
plt.imshow(np.log10(out[cdm_map_num][filter_1+1]))
plt.figure(figsize=(32,22))
plt.subplot(1, 3, 1)
plt.title("Original field")
plt.imshow(np.log10(maps[cdm_map_num]))
plt.subplot(1, 3, 2)
plt.title("Wavelet filter")
plt.imshow(np.fft.fftshift(np.fft.ifft2(scatteringBase.scattering.psi[filter_2][0].squeeze().cpu().detach().numpy())).real)
plt.subplot(1, 3, 3)
plt.title("First order convolved field")
plt.imshow(np.log10(out[cdm_map_num][filter_2+1]))

In [ ]:
plt.figure(figsize=(32,22))
plt.subplot(1, 3, 1)
plt.title("First order 1 field")
plt.imshow(np.log10(out[cdm_map_num][filter_1+1]))
plt.subplot(1, 3, 2)
plt.title("First order 2 field")
plt.imshow(np.log10(out[cdm_map_num][filter_2+1]))
plt.subplot(1, 3, 3)
plt.title("Second order field")
plt.imshow(np.log10(out[cdm_map_num][order2_index]))

In [ ]:
scatteringBase.scattering.out_type